In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import sys

sys.path.append(os.path.join(os.path.dirname("__file__"), "../"))

import warnings

warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

## Data Loading

In [ ]:
iterations = 5
ori_data = np.load(
    "../OUTPUT/stocks-sample-timewak-24/samples/stock_norm_truth_24_test.npy"  # Replace this
)
fake_data = np.load(
    "../OUTPUT/stocks-sample-timewak-24/ddpm_fake_norm_stocks-sample-timewak-24.npy"  # Replace this
)

## Discriminative Score

- To evaluate the classification accuracy between original and synthetic data using post-hoc RNN network. The output is | classification accuracy - 0.5 |.

In [ ]:
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(
        ori_data[:], fake_data[: ori_data.shape[0]]
    )
    discriminative_score.append(temp_disc)
    print(f"Iter {i}: Discriminative = {temp_disc}")

mean, sigma = display_scores(discriminative_score)
print(f"Final Score: {mean:.3f} \xb1 {sigma:.3f}")

## Predictive Score

- To evaluate the prediction performance on train on synthetic, test on real setting. More specifically, we use Post-hoc RNN architecture to predict one-step ahead and report the performance in terms of MAE. 

- The model learns to predict the last dimension with one more step.

In [ ]:
predictive_score = []

for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[: ori_data.shape[0]])
    predictive_score.append(temp_pred)
    print(f"Iter {i}: Predictive = {temp_pred}")

mean, sigma = display_scores(predictive_score)
print(f"Final Score: {mean:.3f} \xb1 {sigma:.3f}")